In [1]:
from collections import Counter
from sklearn.model_selection import KFold
import numpy as np
import re
import os
import shutil

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Bidirectional, Dense, Input, Masking, Lambda
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback, LearningRateScheduler, Callback

Using TensorFlow backend.


In [3]:
TRAIN_FILENAME = "names_and_rubrics_learn.tsv"
TEST_FILENAME = "names_and_rubrics_test_no_rubric.tsv"
MIN_FREQ = 5

TOKENS_NUM = 150767
HIDDEN_DIM = 128
MAX_LEN = 45
LSTM_NUM = 2
HIDDEN_LAYESR_NUM = 2
BATCH_SIZE = 1025
LABELS_NUM = 1222
ACTIVATION = 'tanh'

In [4]:
name_input = Input(shape=(MAX_LEN,))  # shape: (BATCH_SIZE, MAX_LEN)
masked = Masking(mask_value=0)(name_input)
encoded = Embedding(TOKENS_NUM, HIDDEN_DIM)(masked)  # shape: (BATCH_SIZE, MAX_LEN, HIDDEN_DIM)
for i in range(LSTM_NUM - 1):
    encoded = Bidirectional(LSTM(HIDDEN_DIM, return_sequences=True))(encoded)  # shape: (BATCH_SIZE, HIDDEN_DIM)
hidden = Bidirectional(LSTM(HIDDEN_DIM))(encoded)  # shape: (BATCH_SIZE, HIDDEN_DIM)
for i in range(HIDDEN_LAYESR_NUM):
    hidden = Dense(HIDDEN_DIM, activation=ACTIVATION)(hidden)  # shape: (BATCH_SIZE, HIDDEN_DIM)

for i in range(HIDDEN_LAYESR_NUM - 1):
    hidden = Dense(HIDDEN_DIM, activation=ACTIVATION)(hidden)  # shape: (BATCH_SIZE, HIDDEN_DIM)
scores = Dense(LABELS_NUM, activation='softmax')(hidden)  # shape: (BATCH_SIZE, LABELS_NUM)

classificaton_model = Model(inputs=name_input, outputs=scores)

In [5]:
classificaton_model.compile(
    Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
classificaton_model.load_weights("logs_classification_lr=0.0001/epoch_3_batch_7620.h5py")

OSError: Unable to read attribute (len not positive after adjustment for EOA)